In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import scipy.stats as stats

pd.options.mode.chained_assignment = None  
import seaborn as sns


In [ ]:
df_full = pd.read_csv('master_06_FEB_2024.csv')#48873
len(df_full)

In [ ]:
df_full['Z-Scores_ntib'] = stats.zscore(df_full['nocturnal_time_in_bed_period'])
df_full

In [ ]:
df_full[['nocturnal_time_in_bed_duration','nocturnal_time_in_bed_period','sleep_duration','sleep_period']].max()

In [ ]:
df_full = df_full[abs(df_full['Z-Scores_ntib']<4)] 
# df_full = df_full[df_full['nocturnal_time_in_bed_duration']<24] 
len(df_full)

In [ ]:
count_onset = df_full[['participant_id','tib_onset']].groupby(['tib_onset']).count().sort_values(by='participant_id', ascending=False).reset_index()
count_onset.head(3)

In [ ]:
df_full = df_full[df_full['tib_onset']!='2022-04-06 04:23:00+01:00']
len(df_full)

In [ ]:
count_offset = df_full[['participant_id','tib_offset']].groupby(['tib_offset']).count().sort_values(by='participant_id', ascending=False).reset_index()
count_offset.head(3)

In [ ]:
df_full = df_full[df_full['tib_offset']!='2022-04-04 06:58:00+01:00']
df_full = df_full[df_full['tib_offset']!='2022-04-04 06:57:00+01:00']
len(df_full)

In [ ]:
df_full[['participant_id','date','nocturnal_time_in_bed_duration','hours_over_500_lounge', 'before_ntib_mean_light_bedroom',
    'ntib_mean_temperature_bedroom','month', 'bimonth', 'metereological_season']].info()

In [ ]:
df = df_full[['participant_id','date','nocturnal_time_in_bed_duration','hours_over_500_lounge', 'before_ntib_mean_light_bedroom',
    'ntib_mean_temperature_bedroom','month', 'bimonth', 'metereological_season']].dropna()
df.info()

In [ ]:
seasons = df.groupby(['participant_id','bimonth']).size().reset_index(name='count')
seasons

In [ ]:
id_counts = seasons[seasons['count']>=7]['participant_id'].value_counts().reset_index(name='bimonths')
ids2 = id_counts[id_counts['bimonths']>=5]['index'].tolist()
len(ids2)

In [ ]:
ids2 = sorted(ids2)
print(ids2)

#np.save('ids_seven_days_each_five_bimonth.npy',ids2)

In [ ]:
index_no_null = df_full[['participant_id','date','nocturnal_time_in_bed_duration','hours_over_500_lounge', 'before_ntib_mean_light_bedroom',
    'ntib_mean_temperature_bedroom','month', 'bimonth', 'metereological_season']].dropna().index
selected_id = df_full.loc[index_no_null]
selected_id = selected_id [(selected_id ['participant_id'].isin(ids2))].reset_index(drop=True)
len(selected_id) #26523

In [ ]:
selected_id[['participant_id','date','nocturnal_time_in_bed_duration','hours_over_500_lounge', 'before_ntib_mean_light_bedroom',
    'ntib_mean_temperature_bedroom','month', 'bimonth', 'metereological_season']].info()

In [ ]:
selected_id.groupby(['month']).size().reset_index(name='count')

In [ ]:
selected_id.groupby(['bimonth']).size().reset_index(name='count')

In [ ]:
selected_id.groupby(['metereological_season']).size().reset_index(name='count')

In [ ]:
sorted_ids = selected_id.groupby(['participant_id']).size().reset_index(name='Days Of Data').sort_values(by=['Days Of Data'],ascending=False).reset_index(drop=True)
sorted_ids['new_id'] = sorted_ids.index+1
sorted_ids

In [ ]:
selected_id = selected_id.drop(['min_date', 'time_in_study','cluster_number','cluster_classification','start_dt_total'], axis=1)

initial_dates = selected_id.groupby('participant_id')['date'].min()
initial_dates = initial_dates.to_frame().rename(columns={'date':'min_date'}).reset_index()
selected_id = selected_id.merge(initial_dates, on=['participant_id'],how='left')

selected_id['time_in_study'] = (pd.to_datetime(selected_id['date']) - pd.to_datetime(selected_id['min_date'])).dt.days +1
selected_id = selected_id.merge(sorted_ids[['participant_id','new_id']], on=['participant_id'],how='left')

final_dates = selected_id.groupby('participant_id')['date'].max()
final_dates = final_dates.to_frame().rename(columns={'date':'max_date'}).reset_index()
selected_id = selected_id.merge(final_dates, on=['participant_id'],how='left')

selected_id = selected_id.sort_values(by=['new_id','date'],ascending=True)
selected_id['period_of collection'] = (pd.to_datetime(selected_id['max_date']) - pd.to_datetime(selected_id['min_date'])).dt.days

In [ ]:
selected_id.reset_index(drop=True).to_csv('70_participants_metrics_06_FEB_2024.csv')

In [ ]:
sorted_ids = sorted_ids.merge(selected_id[['participant_id','period_of collection','max_date','min_date']], on=['participant_id'],how='inner')
sorted_ids = sorted_ids.drop_duplicates()

In [ ]:
sorted_ids

In [ ]:
sorted_ids.to_csv('sorted_ids_06_FEB_2024.csv')